# **Guía de como Entrenar el Modelo, Guardarlo en un archivo pickle y como usarlo**

Para transformar el archivo `inside_airbnb_model.py` (que originalmente era un notebook) en un script Python que cargue el archivo `df_optimized.csv`, entrene el modelo v5 y lo guarde en formato pickle, sigue estos pasos:

### **Script Python (`train_model_v5.py`)**
```python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler
import pickle
from pathlib import Path

# --- Configuración de paths ---
processed_data_dir = Path("../data/processed/")
model_dir = Path("../models/")
model_dir.mkdir(parents=True, exist_ok=True)  # Crear directorio si no existe

# --- 1. Cargar datos procesados ---
df_optimized = pd.read_csv(processed_data_dir / "df_optimized.csv")

# --- 2. Transformación logarítmica del target (price) ---
y = np.log1p(df_optimized['price'])

# --- 3. Definir features (X) ---
# Eliminar columnas no relevantes para el modelo
X = df_optimized.drop(columns=['price', 'host_since_year'])  # Ajusta según tu CSV

# --- 4. Escalado numérico (opcional, pero recomendado) ---
numeric_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 'number_of_reviews']
scaler = RobustScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# --- 5. División train-test (para evaluación) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 6. Entrenar modelo Random Forest v5 ---
model_v5 = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=2,
    max_features=0.5,
    max_samples=0.8,
    random_state=42,
    n_jobs=-1
)
model_v5.fit(X_train, y_train)

# --- 7. Evaluación del modelo ---
def print_metrics(y_true, y_pred, dataset_name):
    y_true_exp = np.expm1(y_true)
    y_pred_exp = np.expm1(y_pred)
    r2 = r2_score(y_true_exp, y_pred_exp)
    mae = mean_absolute_error(y_true_exp, y_pred_exp)
    print(f"\n📊 **Métricas para {dataset_name}**")
    print(f"R²: {r2:.4f} | MAE: {mae:.2f} EUR")

y_pred_train = model_v5.predict(X_train)
y_pred_test = model_v5.predict(X_test)
print_metrics(y_train, y_pred_train, "ENTRENAMIENTO")
print_metrics(y_test, y_pred_test, "PRUEBA")

# --- 8. Guardar modelo y scaler en pickle ---
with open(model_dir / "model_v5.pkl", 'wb') as f:
    pickle.dump(model_v5, f)

with open(model_dir / "scaler.pkl", 'wb') as f:
    pickle.dump(scaler, f)

print(f"\n✅ Modelo y scaler guardados en: {model_dir}/")
```

---

### **Explicación Paso a Paso**
1. **Carga de datos**:  
   - Lee el archivo `df_optimized.csv` desde la ruta `../data/processed/`.

2. **Preprocesamiento**:  
   - Aplica transformación logarítmica a `price` para manejar colas largas.  
   - Escala features numéricas con `RobustScaler` (protege contra outliers).

3. **Entrenamiento del modelo**:  
   - Usa los hiperparámetros óptimos del **modelo v5** (`n_estimators=300`, `max_features=0.5`, etc.).  
   - Evalúa con métricas en escala original (EUR).

4. **Exportación a pickle**:  
   - Guarda el modelo entrenado (`model_v5.pkl`) y el scaler (`scaler.pkl`) en la carpeta `../models/`.

---



### **Notas Clave**
- **Ajusta las columnas**: Verifica que `X` no incluya columnas irrelevantes o con leakage (como `log_price`).  
- **Entorno reproducible**: Usa `random_state=42` para resultados consistentes.  
- **Requisitos**: Asegúrate de tener las mismas versiones de librerías (`scikit-learn`, `pandas`, etc.) en entrenamiento y producción.  

Si necesitas adaptar algo específico (como las features usadas), modifica el script según tu caso.

-----

## **Como usar el Modelo**

Basado en el script de entrenamiento, aquí tienes cómo probar correctamente el modelo guardado (`random_forest_v5.pkl`):

### **Script completo para probar el modelo (`test_model.py`)**
```python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from sklearn.preprocessing import RobustScaler

# --- Configuración de paths ---
model_dir = Path("models/")
processed_data_dir = Path("data/processed/")

# --- 1. Cargar modelo y scaler ---
with open(model_dir / "random_forest_v5.pkl", 'rb') as f:
    model = pickle.load(f)
    
with open(model_dir / "scaler.pkl", 'rb') as f:
    scaler = pickle.load(f)

# --- 2. Cargar datos de prueba (o nuevos datos) ---
# Opción A: Usar datos de prueba guardados (si existen)
# df_test = pd.read_csv(processed_data_dir / "test_data.csv")

# Opción B: Crear datos de ejemplo (simulando entrada)
test_data = {
    'accommodates': [2, 4],  # Ejemplo: 2 personas
    'bathrooms': [1.0, 2.0],
    'bedrooms': [1, 2],
    'beds': [1, 3],
    'minimum_nights': [3, 2],
    'number_of_reviews': [10, 25],
    'review_scores_rating': [95, 80],
    'host_is_superhost': [1, 0],  # 1=True, 0=False
    'neighbourhood_density': [0.5, 0.3],
    'has_wifi': [1, 1],
    'has_air_conditioning': [1, 0],
    # Añade todas las features usadas en entrenamiento
}

X_new = pd.DataFrame(test_data)

# --- 3. Preprocesamiento igual que en entrenamiento ---
# Escalar features numéricas
numeric_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 
                   'minimum_nights', 'number_of_reviews']
X_new[numeric_features] = scaler.transform(X_new[numeric_features])

# --- 4. Predecir ---
pred_log = model.predict(X_new)  # Predicciones en escala logarítmica
pred_price = np.expm1(pred_log)  # Convertir a precio real (EUR)

# --- 5. Resultados ---
print("\n🔮 Predicciones:")
for i, price in enumerate(pred_price):
    print(f"Alojamiento {i+1}: ${price:.2f} EUR")

# --- 6. Opcional: Importancia de features ---
if hasattr(model, 'feature_importances_'):
    print("\n📊 Importancia de features:")
    features = X_new.columns
    importances = model.feature_importances_
    for feat, imp in sorted(zip(features, importances), key=lambda x: x[1], reverse=True):
        print(f"{feat}: {imp:.4f}")
```

---

### **¿Qué hace este script?**
1. **Carga el modelo y scaler** desde los archivos `.pkl`.
2. **Prepara datos de prueba**:
   - Puedes cargar un CSV con datos reales o simular datos como en el ejemplo.
3. **Aplica el mismo preprocesamiento**:
   - Escalado numérico con `RobustScaler` (¡igual que en entrenamiento!).
4. **Genera predicciones**:
   - Convierte los resultados de log(price) a EUR con `np.expm1()`.
5. **Muestra resultados**:
   - Precios predichos + importancia de cada feature (útil para debug).

---

### **Ejemplo de salida:**
```
🔮 Predicciones:
Alojamiento 1: $85.50 EUR
Alojamiento 2: $120.30 EUR

📊 Importancia de features:
bedrooms: 0.2543
accommodates: 0.1987
bathrooms: 0.1852
...
```

---

### **Requisitos para ejecutarlo**
1. Estructura de directorios:
   ```
   your_project/
   ├── models/
   │   ├── random_forest_v5.pkl
   │   └── scaler.pkl
   └── test_model.py
   ```

2. Bibliotecas:
   ```bash
   pip install pandas numpy scikit-learn
   ```

---

### **Consejos clave**
1. **Verifica las features**:  
   El DataFrame `X_new` debe tener exactamente las mismas columnas que usaste para entrenar (en mismo orden).

2. **Para producción**:  
   Guarda también la lista de columnas usadas en entrenamiento:
   ```python
   # Durante el entrenamiento:
   with open(model_dir / "feature_columns.pkl", 'wb') as f:
       pickle.dump(list(X_train.columns), f)
   
   # Durante la predicción:
   with open(model_dir / "feature_columns.pkl", 'rb') as f:
       expected_columns = pickle.load(f)
   X_new = X_new[expected_columns]  # Reordena columnas
   ```

3. **Si falla**:  
   Revisa que las versiones de `scikit-learn` sean consistentes entre entrenamiento y predicción.